In [5]:
import dimod
from dwave_qbsolv import QBSolv
import dwavebinarycsp
import neal
import itertools
import random
import time
from collections import OrderedDict

def to_base_ten(sample,module_lenght): 
    a = b = 0
    a_vars = ["a"+str(i) for i in range(int(module_lenght/2))]
    b_vars = ["b"+str(i) for i in range(int(module_lenght/2))]
    for lbl in reversed(a_vars):
        a = (a << 1) | sample[lbl]
    for lbl in reversed(b_vars):
        b = (b << 1) | sample[lbl]
    return a, b

def response_to_dict(sampleset, module_length):
    results_dict = OrderedDict()
    for sample, energy in sampleset.data(['sample', 'energy']):
        a, b = to_base_ten(sample, module_length)
        if (a, b) not in results_dict:
            results_dict[(a, b)] = round(energy, 2)
            
    return results_dict

L10 = [493, 551, 437, 391, 323, 667]
L12 = [2183, 1927, 2623, 2491, 2501, 2257, 2419, 1961, 2773, 2173, 2867, 2279, 3233, 1517, 3599, 2021, 3127, 1739, 2537, 1591, 1763]
L14 = [7313, 6767, 10379, 5609, 9523, 8023, 8383, 8989, 9701, 7597, 8549, 4891, 6497, 7957, 7303, 5183, 7739, 5893, 11413, 8611] 
L16 = [35621, 34579, 33017, 26069, 32111, 38009, 20989, 19519, 27641, 33043, 34189, 29353, 36349, 37627, 23213, 56153, 41347]

P10 = 0
P12 = 0
P14 = 0
P16 = 0

T10 = 0
T12 = 0
T14 = 0
T16 = 0

L = [L10,L12,L14,L16]
P = []
T = []

sampler = neal.SimulatedAnnealingSampler()
module_length = 8

for l in L:
    succes = 0
    nprob = 0
    czas = 0
    module_length = module_length +2    
    while succes<3:        
        for n in l:
            start = time.time()
            csp = dwavebinarycsp.factories.multiplication_circuit(int(module_length/2))
            bqm = dwavebinarycsp.stitch(csp)
            p_vars = ["p"+str(i) for i in range(module_length)]  
            # Convert n from decimal to binary
            fixed_variables = dict(zip(reversed(p_vars), ("{:0"+str(module_length)+"b}").format(n)))
            fixed_variables = {var: int(x) for(var, x) in fixed_variables.items()}
            # Fix product variables
            for var, value in fixed_variables.items():
                bqm.fix_variable(var, value) 
            qubo = dimod.BQM.to_qubo(bqm)
            response = QBSolv().sample_qubo(qubo[0],num_reads=10000)
            results = response_to_dict(response, module_length)
            a,b = list(results.items())[0][0][0],list(results.items())[0][0][1]
            stop = time.time()
            czas = czas+stop-start
            #print(a,b,end=',')
            if n % a ==0 or n % b ==0:
                succes = succes +1
                print("SUKCES!")
        nprob = nprob+len(l)        
    T.append(czas/nprob)
    P.append(succes/nprob)
    
print(T)
print(P)

SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
SUKCES!
[1.36087961991628, 2.432481016431536, 2.8627122223377226, 3.7491616004291064]
[0.25, 0.2857142857142857, 0.01875, 0.016042780748663103]


In [27]:
qubo = {('a0', 'b0'): -3, (1, 1): -1, (0, 1): 2, (2, 2): -1,(0, 2): 2, (3, 3): -1, (0, 3): 2}
dict_bqm = dimod.BQM.from_qubo(qubo)
sampler_exact = dimod.ExactSolver()
sampleset = sampler_exact.sample(dict_bqm)
print(sampleset)

   a0 b0  1  0  2  3 energy num_oc.
37  1  1  1  0  1  1   -6.0       1
26  1  1  1  0  1  0   -5.0       1
34  1  1  0  0  1  1   -5.0       1
58  1  1  1  0  0  1   -5.0       1
5   1  1  1  0  0  0   -4.0       1
29  1  1  0  0  1  0   -4.0       1
61  1  1  0  0  0  1   -4.0       1
2   1  1  0  0  0  0   -3.0       1
13  1  1  0  1  0  0   -3.0       1
36  0  1  1  0  1  1   -3.0       1
38  1  0  1  0  1  1   -3.0       1
39  0  0  1  0  1  1   -3.0       1
10  1  1  1  1  0  0   -2.0       1
18  1  1  0  1  1  0   -2.0       1
24  0  0  1  0  1  0   -2.0       1
25  1  0  1  0  1  0   -2.0       1
27  0  1  1  0  1  0   -2.0       1
32  0  0  0  0  1  1   -2.0       1
33  1  0  0  0  1  1   -2.0       1
35  0  1  0  0  1  1   -2.0       1
50  1  1  0  1  0  1   -2.0       1
56  0  0  1  0  0  1   -2.0       1
57  1  0  1  0  0  1   -2.0       1
59  0  1  1  0  0  1   -2.0       1
4   0  1  1  0  0  0   -1.0       1
6   1  0  1  0  0  0   -1.0       1
7   0  0  1  0  0  0   -1.0 